In [ ]:
import boto3
import numpy as np
from astropy.io import fits
import gzip
import re
from io import BytesIO
import pandas as pd 

In [ ]:
existing_fuv = "fuv_objects.csv"

In [ ]:
fuv_objs = pd.read_csv(existing_fuv)
fuv_objs['eclipse']

In [ ]:
fuv_objs['group'] = (fuv_objs.index // 500) + 1
fuv_groups = fuv_objs.groupby('group')

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3')

def download_fits(bucket_name, key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
            return fits.open(BytesIO(f.read()))    
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
        
# Bucket name
bucket_name = 'backplanetest'

for group in fuv_groups:
    # Initialize an array to hold the sum of pixel values
    sum_image = None
    print(group)
    # Add the pixel values of each image
    for e in group[1]['eclipse']:
        i = str(e).zfill(5)
        key = f"e{i}-fd/e{i}-fd-b00-f3000-t00000-g_dose.fits.gz"        
        image = download_fits(bucket_name, key)
        print(key)
        if image is not None:
            if sum_image is None:
                sum_image = np.zeros_like(image[0].data, dtype=np.float64)
            sum_image += image[0].data.astype(np.float64)

    # Save the summed image as a FITS file
    fits.writeto(f'/media/bekah/BekahA/hotspot/{i}_f500_image.fits', sum_image, overwrite=True)
    print(f"Summed image saved as '{i}_g100_image.fits'")

In [ ]:
s3 get_object costs .0004 per 1000 requests, so 6000 eclipses is ~.0012 